### xgb와 als 병합

- xgbranker_recommendations.csv는 파일이 80메가 정도라서 올리지 않겠습니다.
- user_id 와 item_id_list로 구성되어 있습니다.
- user_id 하나당 추천된 item_id가 list형태로 들어가 있습니다.
- 그래서 xgbranker_recommendations.csv의 행의 갯수는 전체 데이터의 user_id의 갯수(638257)와 같습니다.

In [7]:
import pandas as pd
# xgb 데이터프레임의 예시
xgb = pd.DataFrame({
    'user_id': [1, 2],
    'item_id_list': [[101, 102], [201, 202, 203, 204]]
})

In [8]:
xgb

,user_id,item_id_list
0,1,"[101, 102]"
1,2,"[201, 202, 203, 204]"


In [5]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import ast

xgb = pd.read_csv('xgbranker_recommendations.csv')
als = pd.read_csv('../output/output.csv')

# 문자열로 저장된 리스트를 실제 리스트로 변환하는 함수
def convert_to_list(item):
    try:
        # 문자열을 실제 리스트로 변환
        return ast.literal_eval(item) if isinstance(item, str) else item
    except (ValueError, SyntaxError):
        # 변환에 실패한 경우는 그대로 반환
        return item

# item_id_list 열을 실제 리스트로 변환
xgb['item_id_list'] = xgb['item_id_list'].apply(convert_to_list)

# 1. xgb와 als 데이터 병합 (user_id 기준)
# user_id를 기준으로 xgb의 item_id_list를 als와 병합
merged_df = als.merge(xgb, on='user_id', how='left')

# 병합된 데이터를 통해 10개 채우기
def process_user_group(group):
    # item_id_list가 없는 경우 빈 리스트로 대체
    item_id_list = group['item_id_list'].iloc[0]
    # if pd.isnull(item_id_list):
    #     item_id_list = []
    
    # 중복 제외 후 나머지 아이템 채우기
    missing_items = [item for item in group['item_id'].tolist() if item not in item_id_list]
    
    # 10개 채우기
    combined_list = item_id_list + missing_items[:10 - len(item_id_list)]
    
    # 결과 저장
    result = [{'user_id': group['user_id'].iloc[0], 'item_id': item} for item in combined_list]
    return result

# 병렬 처리로 각 user_id에 대한 작업 수행
def parallel_process(df, max_workers=8):
    results = []
    user_groups = df.groupby('user_id')
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # tqdm을 사용하여 진행 상황 표시
        futures = [executor.submit(process_user_group, group) for _, group in tqdm(user_groups,desc='future')]
        for future in tqdm(futures, desc="Processing user_id"):
            results.extend(future.result())
    
    return pd.DataFrame(results)

# 병렬 처리 실행
result_df = parallel_process(merged_df, max_workers=32)
result_df.to_csv('xgb_als_output.csv',index=False)
# 출력
print(result_df)


Processing user_id: 100%|██████████| 638257/638257 [02:06<00:00, 5062.47it/s]


                                      user_id  \
0        0000162d-72b9-4b0f-8895-7b52b6661c82   
1        0000162d-72b9-4b0f-8895-7b52b6661c82   
2        0000162d-72b9-4b0f-8895-7b52b6661c82   
3        0000162d-72b9-4b0f-8895-7b52b6661c82   
4        0000162d-72b9-4b0f-8895-7b52b6661c82   
...                                       ...   
6382565  ffffc959-2575-476c-b897-057e51182c5f   
6382566  ffffc959-2575-476c-b897-057e51182c5f   
6382567  ffffc959-2575-476c-b897-057e51182c5f   
6382568  ffffc959-2575-476c-b897-057e51182c5f   
6382569  ffffc959-2575-476c-b897-057e51182c5f   

                                      item_id  
0        a9e8c988-8b93-4ea2-8a97-b9ef898342dd  
1        0bb05529-df13-46ca-8bfc-832344a7b1d3  
2        383fde85-a76b-447d-ab42-0e26e4897420  
3        1dc96fe9-67b0-4088-90f4-61f5c505091d  
4        68baa02f-ad74-4399-b0c9-1f0bf326fc92  
...                                       ...  
6382565  f8638620-f4e0-499f-b055-33d1fa8825e6  
6382566  6727e2bf-149d-4c40

In [1]:
# expanded_xgb = xgb.explode('item_id_list').rename(columns={'item_id_list': 'item_id'})
# expanded_xgb[:30]